# Create synthetic data from a previously archived model

This blueprint will show you how to download a previously created synthetic data model, which can be saved to a `tar.gz` file after training. From here, you can download the model and easily generate new data.

In [ ]:
%%capture

# The test model below requires > v0.14.0 synthetics and was built with TF 2.3.1

!pip install -U smart-open gretel-client "gretel-synthetics>=0.14.0"

In [ ]:
# Where to unpack the archived model to

TARGET_DIR = "my_model_dir"

REMOTE_MODEL = "https://gretel-public-website.s3-us-west-2.amazonaws.com/models/safecast/gretel-synthetics-safecast-model.tar.gz"

In [ ]:
import os
import getpass

API_KEY = os.getenv("GRETEL_API_KEY") or getpass.getpass("Gretel API Key:")

In [ ]:
# Install Gretel SDKs

from gretel_client import get_cloud_client

client = get_cloud_client("api", API_KEY)
client.install_packages()

In [ ]:
from pathlib import Path
import gzip
import tarfile

from smart_open import open as smart_open

def unpack_remote_model_to_dir(remote_file: str, target_dir: str):
    model_path = Path(target_dir)
    if not model_path.is_dir():
        model_path.mkdir()
        
    with smart_open(remote_file, "rb", ignore_ext=True) as fin:
        with gzip.open(fin) as gzip_in:
            with tarfile.open(fileobj=gzip_in, mode="r:gz") as tar_in:
                tar_in.extractall(model_path)

In [ ]:
from gretel_helpers.synthetics import SyntheticDataBundle

def start():
    unpack_remote_model_to_dir(REMOTE_MODEL, TARGET_DIR)
    bundle = SyntheticDataBundle.load(TARGET_DIR)
    bundle.generate(num_lines=100)
    return bundle

In [ ]:
bundle = start()

In [ ]:
bundle.get_synthetic_df().head()